In [12]:
library(dplyr)
library(lubridate)
library(R.matlab)
library(tidyr)
library(tibble)
library(rdbnomics)

ERROR: Error in library(R.matlab): there is no package called 'R.matlab'


In [4]:
gdp_values <- rdb(ids="INSEE/CNT-2010-PIB-EQB-RF/T.CNT-EQUILIBRE_PIB.SO.PIB.SO.VALEUR_ABSOLUE.FE.V.EUROS.CVS-CJO") %>%
add_column(var="gdp_values")

In [5]:
expenditures_values <-
rdb(ids="INSEE/CNT-2010-CSI/T.CNT-OPERATIONS_SECTEURS_INST.S13.E.OTE.VALEUR_ABSOLUE.FE.EUROS.CVS-CJO") %>%
add_column(var="expenditures_values")

In [6]:
debt_gdp <-
rdb(ids="INSEE/DETTE-TRIM-APU/T.DETTE_MAASTRICHT.S13.F.PROPORTION.FE.POURCENT.BRUT.SO") %>%
add_column(var="debt_gdp") %>%
mutate(value=value/100)

In [10]:
df <-
bind_rows(gdp_values,expenditures_values,debt_gdp) %>%
select(period,value,var) %>%
spread(var,value) %>%
mutate(expenditures_gdp=expenditures_values/gdp_values) %>%
select(period,expenditures_gdp,debt_gdp) %>%
filter(year(period)>=1996 & year(period)<=2017) %>%
mutate(period=paste(year(period),quarter(period),sep="Q"))

colnames(df)[1] <- ""

write.csv(df,"data.csv",row.names = F)

In [1]:
system2(
    'octave',
    args = c('--eval', '"addpath /usr/local/lib/dynare/matlab/; dynare model.mod"', '--no-window-system'),
    stdout = "stdout.txt",
    stderr = "stderr.txt",
    stdin = "",
    input = NULL,
    env = character(),
    wait = TRUE,
)
# The output of this command is written in files `stderr.txt` and `stdout.txt`.
# Dynare use the model `model.mod` and generate the files beginning with `model`.